In [1]:
import json
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha
from Artha.extraction import *
from Artha.analysis import *

print("done")

done


In [5]:
with open("../data/tweets/uCryptoKaleotweets.json", "r") as r:
    tweets = json.load(r)
tweet_text = [tweet["full_text"] for tweet in tweets]

In [ ]:
print_polarity(tweet_text[0])

In [6]:
final_ticks, final_names, final_cash = [], [], []

with open("senti.txt", "w+") as w:
    for ind, i in enumerate(tweet_text):
        tick = extract_crypto_tickers(i)
        name = extract_crypto_names(i)
        cash = extract_cashtags(i)

        final_ticks.extend(tick)
        final_names.extend(name)
        final_cash.extend([c[1:] for c in cash])

        if tick: # or name

            json.dump(tick, w)
            w.write("\n")
            vs = analyzer.polarity_scores(i)
            w.write("{:-<65} {}\n\n".format(i, str(vs)))